In [69]:
import pandas as pd
import h5py
import tensorflow as tf
import keras
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

In [50]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
#file=h5py.File("/gdrive/My Drive/SVHN_single_grey1.h5")

In [70]:
file=h5py.File("SVHN_single_grey1.h5")

In [0]:
list(file)

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']

In [71]:
X_test=np.asarray(file["X_test"])

In [72]:
X_train=np.asarray(file["X_train"])

In [73]:
y_train=np.asarray(file["y_train"])

In [74]:
y_test=np.asarray(file["y_test"])

In [75]:
X_train.shape

(42000, 32, 32)

In [0]:
X_test.shape

(18000, 32, 32)

In [76]:
y_train.shape

(42000,)

In [0]:
y_test.shape

(18000,)

In [0]:
X_train[0].shape

(32, 32)

**Using KNN Classifier**

In [0]:
nsamples,nx,ny=X_train.shape
knn_train = X_train.reshape((nsamples,nx*ny))

In [0]:
knn_train.shape

(42000, 1024)

In [0]:
knn_train[0]

array([33.0704, 30.2601, 26.852 , ..., 49.6682, 50.853 , 53.0377],
      dtype=float32)

In [0]:
nsamples,nx,ny=X_test.shape
knn_test=X_test.reshape((nsamples,nx*ny))

In [0]:
knn_test.shape

(18000, 1024)

In [0]:
knn_test[0]

array([ 40.558 ,  46.7917,  48.9764, ..., 110.0169, 111.2017, 114.1906],
      dtype=float32)

In [0]:
scaler=StandardScaler()

In [0]:
knn_train_scaled=scaler.fit_transform(knn_train)

In [0]:
knn_test_scaled=scaler.fit_transform(knn_test)

In [0]:
kVals = range(1, 30, 2)


In [0]:
accuracies = []

In [0]:
for k in range(1, 60, 2):
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(knn_train_scaled,y_train)
    knn_predictions=knn_model.predict(knn_test_scaled)
    knn_score = accuracy_score(knn_predictions,y_test)
    print("k=%d, accuracy=%.2f%%" % (k, knn_score * 100))
    accuracies.append(knn_score)

k=1, accuracy=46.77%
k=3, accuracy=47.19%
k=5, accuracy=50.03%
k=7, accuracy=51.52%
k=9, accuracy=52.18%
k=11, accuracy=52.83%
k=13, accuracy=52.97%
k=15, accuracy=53.44%
k=17, accuracy=53.46%
k=19, accuracy=53.76%
k=21, accuracy=54.13%
k=23, accuracy=54.10%
k=25, accuracy=53.98%
k=27, accuracy=54.30%
k=29, accuracy=54.43%
k=31, accuracy=54.52%
k=33, accuracy=54.47%
k=35, accuracy=54.62%
k=37, accuracy=54.81%
k=39, accuracy=54.66%
k=41, accuracy=54.64%
k=43, accuracy=54.93%
k=45, accuracy=54.91%
k=47, accuracy=54.93%
k=49, accuracy=54.85%
k=51, accuracy=54.79%
k=53, accuracy=54.82%
k=55, accuracy=54.72%
k=57, accuracy=54.83%
k=59, accuracy=54.67%


Trying the value of k as square root of training samples: sqrt of 4200 : 205

In [0]:
knn_model = KNeighborsClassifier(n_neighbors=205)


In [0]:
knn_model.fit(knn_train_scaled,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=205, p=2,
                     weights='uniform')

In [0]:
knn_predictions = knn_model.predict(knn_test_scaled)


In [0]:
knn_score =accuracy_score(knn_predictions,y_test)

In [0]:
knn_score

0.5249444444444444

Observations: Even after taking optimal value of k as 209 seems like square root of number of training samples, accuracy has decreased to 51%. So we will proceed with the value of k as 29 as accuracy of model reaches to 53% 

Building final model with k=29 and then calaulating the classification report for the same

In [0]:
final_knn_model=KNeighborsClassifier(n_neighbors=47)

In [0]:
final_knn_model.fit(knn_train_scaled,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=47, p=2,
                     weights='uniform')

In [0]:
final_knn_predictions=final_knn_model.predict(knn_test_scaled)

In [0]:
final_knn_score=accuracy_score(final_knn_predictions,y_test)

In [0]:
final_knn_score

0.5492777777777778

In [0]:
print(classification_report(y_test,knn_predictions))

              precision    recall  f1-score   support

           0       0.41      0.72      0.52      1814
           1       0.43      0.72      0.54      1828
           2       0.65      0.49      0.56      1803
           3       0.49      0.42      0.45      1719
           4       0.66      0.65      0.65      1812
           5       0.54      0.40      0.46      1768
           6       0.54      0.40      0.46      1832
           7       0.67      0.61      0.64      1808
           8       0.52      0.39      0.44      1812
           9       0.55      0.45      0.50      1804

    accuracy                           0.52     18000
   macro avg       0.55      0.52      0.52     18000
weighted avg       0.55      0.52      0.52     18000



**Building Deep Neural Network Model**

In [77]:
#X_train_deep = X_train.reshape(X_train.shape[0], 32, 32, 1).astype('float32')
#X_test_deep = X_test.reshape(X_test.shape[0], 32, 32, 1).astype('float32')
X_train_model =X_train/255
X_test_model  =X_test/255

In [78]:
y_train_model = np_utils.to_categorical(y_train, 10)
y_test_model = np_utils.to_categorical(y_test, 10)


In [79]:
model = tf.keras.models.Sequential()


In [80]:
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32,)))


In [81]:
model.add(tf.keras.layers.BatchNormalization())

In [82]:
model.add(tf.keras.layers.Dense(200, activation='relu'))

In [83]:
model.add(tf.keras.layers.Dense(100, activation='relu'))


In [84]:
model.add(tf.keras.layers.Dense(60, activation='relu'))


In [85]:
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [86]:
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03, decay=0.001)


In [87]:
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
model.fit(X_train_model,y_train_model,          
          validation_data=(X_test_model,y_test_model),
          epochs=300,
          batch_size=32)

Train on 42000 samples, validate on 18000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
42000/42000 [==============================] - 7s 177us/sample - loss: 1.4605 - acc: 0.5142 - val_loss: 1.0302 - val_acc: 0.6818
Epoch 2/300
42000/42000 [==============================] - 7s 171us/sample - loss: 0.9798 - acc: 0.6947 - val_loss: 0.8574 - val_acc: 0.7394
Epoch 3/300
42000/42000 [==============================] - 12s 296us/sample - loss: 0.8490 - acc: 0.7374 - val_loss: 0.7784 - val_acc: 0.7694
Epoch 4/300
42000/42000 [==============================] - 11s 257us/sample - loss: 0.7775 - acc: 0.7599 - val_loss: 0.7355 - val_acc: 0.7857
Epoch 5/300
42000/42000 [==============================] - 9s 216us/sample - loss: 0.7341 - acc: 0.7727 - val_loss: 0.6972 - val_acc: 0.7992
Epoch 6/300
42000/42000 [==============================] - 12s 274us/sample - loss: 0.7004 - acc: 0.7845 - val_loss: 0.6691 - val_acc: 0.8073
Epoch 7/300
42000/42000 [============================

42000/42000 [==============================] - 7s 161us/sample - loss: 0.4516 - acc: 0.8597 - val_loss: 0.5568 - val_acc: 0.8488
Epoch 57/300
42000/42000 [==============================] - 7s 159us/sample - loss: 0.4545 - acc: 0.8588 - val_loss: 0.5554 - val_acc: 0.8482
Epoch 58/300
42000/42000 [==============================] - 7s 160us/sample - loss: 0.4509 - acc: 0.8614 - val_loss: 0.5557 - val_acc: 0.8490
Epoch 59/300
42000/42000 [==============================] - 7s 159us/sample - loss: 0.4491 - acc: 0.8617 - val_loss: 0.5555 - val_acc: 0.8497
Epoch 60/300
42000/42000 [==============================] - 7s 158us/sample - loss: 0.4514 - acc: 0.8605 - val_loss: 0.5572 - val_acc: 0.8486
Epoch 61/300
42000/42000 [==============================] - 7s 161us/sample - loss: 0.4471 - acc: 0.8632 - val_loss: 0.5525 - val_acc: 0.8491
Epoch 62/300
42000/42000 [==============================] - 7s 158us/sample - loss: 0.4485 - acc: 0.8615 - val_loss: 0.5550 - val_acc: 0.8495
Epoch 63/300
42000/

Epoch 114/300
42000/42000 [==============================] - 7s 158us/sample - loss: 0.4148 - acc: 0.8719 - val_loss: 0.5451 - val_acc: 0.8544
Epoch 115/300
42000/42000 [==============================] - 7s 165us/sample - loss: 0.4124 - acc: 0.8713 - val_loss: 0.5450 - val_acc: 0.8554
Epoch 116/300
42000/42000 [==============================] - 7s 160us/sample - loss: 0.4105 - acc: 0.8717 - val_loss: 0.5462 - val_acc: 0.8551
Epoch 117/300
42000/42000 [==============================] - 7s 158us/sample - loss: 0.4120 - acc: 0.8717 - val_loss: 0.5459 - val_acc: 0.8551
Epoch 118/300
42000/42000 [==============================] - 7s 161us/sample - loss: 0.4090 - acc: 0.8736 - val_loss: 0.5443 - val_acc: 0.8561
Epoch 119/300
42000/42000 [==============================] - 7s 163us/sample - loss: 0.4082 - acc: 0.8741 - val_loss: 0.5452 - val_acc: 0.8554
Epoch 120/300
42000/42000 [==============================] - 7s 161us/sample - loss: 0.4100 - acc: 0.8729 - val_loss: 0.5438 - val_acc: 0.8557

42000/42000 [==============================] - 9s 213us/sample - loss: 0.3916 - acc: 0.8785 - val_loss: 0.5437 - val_acc: 0.8573
Epoch 172/300
42000/42000 [==============================] - 9s 209us/sample - loss: 0.3911 - acc: 0.8769 - val_loss: 0.5419 - val_acc: 0.8559
Epoch 173/300
42000/42000 [==============================] - 7s 168us/sample - loss: 0.3913 - acc: 0.8771 - val_loss: 0.5408 - val_acc: 0.8577
Epoch 174/300
42000/42000 [==============================] - 8s 193us/sample - loss: 0.3948 - acc: 0.8768 - val_loss: 0.5423 - val_acc: 0.8570
Epoch 175/300
42000/42000 [==============================] - 7s 162us/sample - loss: 0.3923 - acc: 0.8781 - val_loss: 0.5418 - val_acc: 0.8576
Epoch 176/300
42000/42000 [==============================] - 7s 175us/sample - loss: 0.3928 - acc: 0.8774 - val_loss: 0.5421 - val_acc: 0.8571
Epoch 177/300
42000/42000 [==============================] - 7s 163us/sample - loss: 0.3888 - acc: 0.8765 - val_loss: 0.5428 - val_acc: 0.8569
Epoch 178/300

42000/42000 [==============================] - 7s 165us/sample - loss: 0.3767 - acc: 0.8820 - val_loss: 0.5411 - val_acc: 0.8577
Epoch 229/300
42000/42000 [==============================] - 7s 162us/sample - loss: 0.3810 - acc: 0.8804 - val_loss: 0.5422 - val_acc: 0.8570
Epoch 230/300
42000/42000 [==============================] - 7s 162us/sample - loss: 0.3778 - acc: 0.8803 - val_loss: 0.5397 - val_acc: 0.8576
Epoch 231/300
42000/42000 [==============================] - 7s 170us/sample - loss: 0.3757 - acc: 0.8832 - val_loss: 0.5416 - val_acc: 0.8578
Epoch 232/300
42000/42000 [==============================] - 7s 177us/sample - loss: 0.3756 - acc: 0.8831 - val_loss: 0.5432 - val_acc: 0.8576
Epoch 233/300
42000/42000 [==============================] - 7s 167us/sample - loss: 0.3796 - acc: 0.8815 - val_loss: 0.5413 - val_acc: 0.8576
Epoch 234/300
42000/42000 [==============================] - 7s 169us/sample - loss: 0.3735 - acc: 0.8845 - val_loss: 0.5423 - val_acc: 0.8569
Epoch 235/300

42000/42000 [==============================] - 7s 157us/sample - loss: 0.3711 - acc: 0.8834 - val_loss: 0.5407 - val_acc: 0.8579
Epoch 286/300
42000/42000 [==============================] - 7s 164us/sample - loss: 0.3651 - acc: 0.8855 - val_loss: 0.5401 - val_acc: 0.8578
Epoch 287/300
42000/42000 [==============================] - 7s 160us/sample - loss: 0.3682 - acc: 0.8863 - val_loss: 0.5423 - val_acc: 0.8580
Epoch 288/300
42000/42000 [==============================] - 7s 166us/sample - loss: 0.3656 - acc: 0.8849 - val_loss: 0.5427 - val_acc: 0.8588
Epoch 289/300
42000/42000 [==============================] - 7s 162us/sample - loss: 0.3658 - acc: 0.8863 - val_loss: 0.5412 - val_acc: 0.8586
Epoch 290/300
42000/42000 [==============================] - 7s 162us/sample - loss: 0.3664 - acc: 0.8853 - val_loss: 0.5414 - val_acc: 0.8581
Epoch 291/300
42000/42000 [==============================] - 7s 165us/sample - loss: 0.3690 - acc: 0.8849 - val_loss: 0.5422 - val_acc: 0.8577
Epoch 292/300

In [63]:
model_predictions=model.predict_classes(X_test)

In [67]:
model_predictions[1]

7

In [88]:
y_test[0]

1

In [89]:
print(confusion_matrix(y_test,model_predictions))

[[1637   24    9   14   14    4   23   22   23   44]
 [  27 1603   22   29   46    7   12   47   21   14]
 [  17   22 1578   39   21   14    5   46   28   33]
 [  21   36   31 1397   19  107   16   34   37   21]
 [  21   53   24   23 1603    9   19   13   12   35]
 [  15   10   11   86   12 1510   46   11   41   26]
 [  43   23   10   24   39   62 1531    9   71   20]
 [  14   64   31   28   13    9    9 1611   12   17]
 [  37   38   24   54   12   30   76    9 1483   49]
 [  64   37   28   39   19   47   12   21   39 1498]]


In [90]:
print(classification_report(y_test,model_predictions))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1814
           1       0.84      0.88      0.86      1828
           2       0.89      0.88      0.88      1803
           3       0.81      0.81      0.81      1719
           4       0.89      0.88      0.89      1812
           5       0.84      0.85      0.85      1768
           6       0.88      0.84      0.86      1832
           7       0.88      0.89      0.89      1808
           8       0.84      0.82      0.83      1812
           9       0.85      0.83      0.84      1804

   micro avg       0.86      0.86      0.86     18000
   macro avg       0.86      0.86      0.86     18000
weighted avg       0.86      0.86      0.86     18000



# Implementing the complete network using Tensorflow: Manually Creating Layers 

In [3]:
session=tf.Session()

In [4]:
X_train=np.asarray(file["X_train"])
X_test=np.asarray(file["X_test"])

In [5]:
X_train /= 255
X_test /= 255

In [6]:
y_train=np.asarray(file["y_train"])
y_test=np.asarray(file["y_test"])

In [7]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [8]:
n_input_shape=32*32
n_classes=10

In [9]:
X=tf.placeholder(tf.float32,shape=[None,n_input_shape],name="X")
y=tf.placeholder(tf.float32,shape=[None,n_classes],name="y")

In [10]:
def weight_variable(name,shape):
    initer=tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable("W_"+name,dtype=tf.float32,shape=shape,initializer=initer)

In [11]:
def bias(name,shape):
    initial=tf.constant(0.,shape=shape,dtype=tf.float32)
    return tf.get_variable("b_"+name,dtype=tf.float32,initializer=initial)

In [12]:
def fc_layer(x,nodes,name,use_relu=True):
    in_dim=x.get_shape()[1]
    w=weight_variable(name,shape=[in_dim,nodes])
    b=bias(name,[nodes])
    layer=tf.add(tf.matmul(x,w),b)
    if use_relu:
        layer=tf.nn.relu(layer)
    return layer        

In [13]:
fc1=fc_layer(X,100,"layer1",use_relu=True)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
fc2=fc_layer(fc1,200,"layer2",use_relu=True)

In [15]:
fc3=fc_layer(fc2,150,"layer3",use_relu=True)

In [16]:
fc4=fc_layer(fc3,60,"layer4",use_relu=True)

In [17]:
output_layer=fc_layer(fc4,n_classes,"output",use_relu=False)

In [18]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=output_layer))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [19]:
correct_prediction=tf.equal(tf.argmax(output_layer,1),tf.argmax(y,1))

In [20]:
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [34]:
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.5)

In [22]:
train=optimizer.minimize(loss)

In [23]:
init=tf.global_variables_initializer()

In [24]:
session.run(init)    

In [25]:
num_pixels = X_train.shape[1] * X_train.shape[2]


In [26]:
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')


In [27]:
X_train.shape

(42000, 1024)

In [28]:
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')


In [29]:
X_test.shape

(18000, 1024)

In [30]:
y_train.shape

(42000, 10)

In [31]:
y_test.shape

(18000, 10)

In [35]:
for i in range(1,2000):
    random_indices=np.random.choice(X_train.shape[0],50)
    #print(random_indices)
    x_train_data=X_train[random_indices]
    y_train_data=y_train[random_indices]
    test_random_indices=np.random.choice(X_test.shape[0],50)
    x_test_data=X_test[test_random_indices]
    y_test_data=y_test[test_random_indices]
    session.run(train,feed_dict={X:x_train_data,y:y_train_data})
    print("TrainLoss:",session.run(loss,feed_dict={X:x_train_data,y:y_train_data}),\
         "TrainAcc:",session.run(accuracy,feed_dict={X:x_train_data,y:y_train_data}),\
         "TestLoss:",session.run(loss,feed_dict={X:x_test_data,y:y_test_data}),\
         "TestAcc:",session.run(accuracy,feed_dict={X:x_test_data,y:y_test_data}))
   

TrainLoss: 2.30193 TrainAcc: 0.12 TestLoss: 2.3031788 TestAcc: 0.12
TrainLoss: 2.301789 TrainAcc: 0.08 TestLoss: 2.302671 TestAcc: 0.16
TrainLoss: 2.3023276 TrainAcc: 0.06 TestLoss: 2.3026538 TestAcc: 0.08
TrainLoss: 2.3025098 TrainAcc: 0.1 TestLoss: 2.298702 TestAcc: 0.18
TrainLoss: 2.3025777 TrainAcc: 0.1 TestLoss: 2.3019164 TestAcc: 0.04
TrainLoss: 2.3038473 TrainAcc: 0.04 TestLoss: 2.2996626 TestAcc: 0.1
TrainLoss: 2.304449 TrainAcc: 0.06 TestLoss: 2.305266 TestAcc: 0.04
TrainLoss: 2.3036685 TrainAcc: 0.08 TestLoss: 2.3018804 TestAcc: 0.12
TrainLoss: 2.3008163 TrainAcc: 0.18 TestLoss: 2.3024664 TestAcc: 0.14
TrainLoss: 2.302521 TrainAcc: 0.1 TestLoss: 2.30062 TestAcc: 0.12
TrainLoss: 2.3029077 TrainAcc: 0.08 TestLoss: 2.302767 TestAcc: 0.1
TrainLoss: 2.3045735 TrainAcc: 0.06 TestLoss: 2.3033228 TestAcc: 0.12
TrainLoss: 2.3035517 TrainAcc: 0.14 TestLoss: 2.3005548 TestAcc: 0.2
TrainLoss: 2.3011057 TrainAcc: 0.12 TestLoss: 2.299236 TestAcc: 0.24
TrainLoss: 2.300847 TrainAcc: 0.1 Test

TrainLoss: 2.3017454 TrainAcc: 0.04 TestLoss: 2.3053129 TestAcc: 0.02
TrainLoss: 2.300864 TrainAcc: 0.1 TestLoss: 2.3071609 TestAcc: 0.06
TrainLoss: 2.301652 TrainAcc: 0.14 TestLoss: 2.3037531 TestAcc: 0.02
TrainLoss: 2.3025146 TrainAcc: 0.08 TestLoss: 2.3029609 TestAcc: 0.12
TrainLoss: 2.3027527 TrainAcc: 0.14 TestLoss: 2.3034928 TestAcc: 0.06
TrainLoss: 2.3011231 TrainAcc: 0.12 TestLoss: 2.3026924 TestAcc: 0.06
TrainLoss: 2.302585 TrainAcc: 0.1 TestLoss: 2.3050692 TestAcc: 0.06
TrainLoss: 2.3028197 TrainAcc: 0.14 TestLoss: 2.3013809 TestAcc: 0.2
TrainLoss: 2.3023715 TrainAcc: 0.06 TestLoss: 2.3063805 TestAcc: 0.04
TrainLoss: 2.2990081 TrainAcc: 0.16 TestLoss: 2.3002279 TestAcc: 0.12
TrainLoss: 2.3053193 TrainAcc: 0.04 TestLoss: 2.304141 TestAcc: 0.06
TrainLoss: 2.3004823 TrainAcc: 0.12 TestLoss: 2.3025596 TestAcc: 0.1
TrainLoss: 2.3041573 TrainAcc: 0.1 TestLoss: 2.3055758 TestAcc: 0.06
TrainLoss: 2.3052158 TrainAcc: 0.08 TestLoss: 2.2974322 TestAcc: 0.18
TrainLoss: 2.3006122 TrainAcc

TrainLoss: 2.3061182 TrainAcc: 0.04 TestLoss: 2.3052778 TestAcc: 0.06
TrainLoss: 2.3011649 TrainAcc: 0.16 TestLoss: 2.3050857 TestAcc: 0.04
TrainLoss: 2.2996418 TrainAcc: 0.14 TestLoss: 2.3031275 TestAcc: 0.16
TrainLoss: 2.2994783 TrainAcc: 0.12 TestLoss: 2.3025854 TestAcc: 0.12
TrainLoss: 2.3002768 TrainAcc: 0.14 TestLoss: 2.3031123 TestAcc: 0.1
TrainLoss: 2.3038893 TrainAcc: 0.06 TestLoss: 2.302474 TestAcc: 0.12
TrainLoss: 2.3029346 TrainAcc: 0.1 TestLoss: 2.3057663 TestAcc: 0.1
TrainLoss: 2.3024337 TrainAcc: 0.14 TestLoss: 2.3011482 TestAcc: 0.12
TrainLoss: 2.299629 TrainAcc: 0.08 TestLoss: 2.3039355 TestAcc: 0.06
TrainLoss: 2.301599 TrainAcc: 0.1 TestLoss: 2.2997324 TestAcc: 0.22
TrainLoss: 2.301552 TrainAcc: 0.08 TestLoss: 2.3055964 TestAcc: 0.02
TrainLoss: 2.3014228 TrainAcc: 0.12 TestLoss: 2.3038123 TestAcc: 0.08
TrainLoss: 2.3017514 TrainAcc: 0.12 TestLoss: 2.3021286 TestAcc: 0.08
TrainLoss: 2.304114 TrainAcc: 0.04 TestLoss: 2.3002388 TestAcc: 0.1
TrainLoss: 2.3034494 TrainAcc:

TrainLoss: 2.3029644 TrainAcc: 0.08 TestLoss: 2.3008785 TestAcc: 0.12
TrainLoss: 2.3008273 TrainAcc: 0.12 TestLoss: 2.3010933 TestAcc: 0.08
TrainLoss: 2.30391 TrainAcc: 0.04 TestLoss: 2.3041282 TestAcc: 0.06
TrainLoss: 2.299287 TrainAcc: 0.16 TestLoss: 2.3043349 TestAcc: 0.08
TrainLoss: 2.300955 TrainAcc: 0.08 TestLoss: 2.3034284 TestAcc: 0.04
TrainLoss: 2.302041 TrainAcc: 0.02 TestLoss: 2.3013556 TestAcc: 0.12
TrainLoss: 2.302963 TrainAcc: 0.12 TestLoss: 2.305487 TestAcc: 0.08
TrainLoss: 2.3005967 TrainAcc: 0.12 TestLoss: 2.299915 TestAcc: 0.14
TrainLoss: 2.3033946 TrainAcc: 0.02 TestLoss: 2.3023992 TestAcc: 0.14
TrainLoss: 2.3009677 TrainAcc: 0.1 TestLoss: 2.3051689 TestAcc: 0.1
TrainLoss: 2.3047793 TrainAcc: 0.1 TestLoss: 2.301249 TestAcc: 0.04
TrainLoss: 2.3013604 TrainAcc: 0.12 TestLoss: 2.3030493 TestAcc: 0.06
TrainLoss: 2.3007221 TrainAcc: 0.08 TestLoss: 2.305031 TestAcc: 0.06
TrainLoss: 2.3018346 TrainAcc: 0.14 TestLoss: 2.3028252 TestAcc: 0.12
TrainLoss: 2.3013058 TrainAcc: 0.

TrainLoss: 2.2963226 TrainAcc: 0.18 TestLoss: 2.303818 TestAcc: 0.08
TrainLoss: 2.3009408 TrainAcc: 0.08 TestLoss: 2.302812 TestAcc: 0.12
TrainLoss: 2.3034556 TrainAcc: 0.1 TestLoss: 2.3033082 TestAcc: 0.08
TrainLoss: 2.3028548 TrainAcc: 0.06 TestLoss: 2.3055835 TestAcc: 0.1
TrainLoss: 2.3016176 TrainAcc: 0.12 TestLoss: 2.3029094 TestAcc: 0.08
TrainLoss: 2.303773 TrainAcc: 0.08 TestLoss: 2.3048391 TestAcc: 0.04
TrainLoss: 2.3043516 TrainAcc: 0.08 TestLoss: 2.3026009 TestAcc: 0.08
TrainLoss: 2.3017623 TrainAcc: 0.12 TestLoss: 2.3028603 TestAcc: 0.06
TrainLoss: 2.3014748 TrainAcc: 0.08 TestLoss: 2.3031943 TestAcc: 0.14
TrainLoss: 2.3033674 TrainAcc: 0.1 TestLoss: 2.303956 TestAcc: 0.08
TrainLoss: 2.3024511 TrainAcc: 0.14 TestLoss: 2.3057702 TestAcc: 0.04
TrainLoss: 2.302988 TrainAcc: 0.08 TestLoss: 2.3021374 TestAcc: 0.06
TrainLoss: 2.301902 TrainAcc: 0.12 TestLoss: 2.3026545 TestAcc: 0.06
TrainLoss: 2.3023179 TrainAcc: 0.08 TestLoss: 2.303395 TestAcc: 0.1
TrainLoss: 2.3020327 TrainAcc: 

...

TrainLoss: 2.3025732 TrainAcc: 0.12 TestLoss: 2.305032 TestAcc: 0.04
TrainLoss: 2.3044598 TrainAcc: 0.08 TestLoss: 2.3050613 TestAcc: 0.06
TrainLoss: 2.3018885 TrainAcc: 0.08 TestLoss: 2.3014734 TestAcc: 0.1
TrainLoss: 2.303343 TrainAcc: 0.1 TestLoss: 2.3048313 TestAcc: 0.08
TrainLoss: 2.299552 TrainAcc: 0.16 TestLoss: 2.305404 TestAcc: 0.06
TrainLoss: 2.3038526 TrainAcc: 0.08 TestLoss: 2.2983625 TestAcc: 0.2
TrainLoss: 2.30223 TrainAcc: 0.1 TestLoss: 2.301507 TestAcc: 0.08
TrainLoss: 2.3049195 TrainAcc: 0.08 TestLoss: 2.3006897 TestAcc: 0.1
TrainLoss: 2.3009734 TrainAcc: 0.08 TestLoss: 2.302408 TestAcc: 0.06
TrainLoss: 2.3029578 TrainAcc: 0.1 TestLoss: 2.3040614 TestAcc: 0.06
TrainLoss: 2.3018374 TrainAcc: 0.14 TestLoss: 2.300452 TestAcc: 0.14
TrainLoss: 2.3039813 TrainAcc: 0.04 TestLoss: 2.3014116 TestAcc: 0.1
TrainLoss: 2.3019097 TrainAcc: 0.1 TestLoss: 2.3010612 TestAcc: 0.12
TrainLoss: 2.3025181 TrainAcc: 0.08 TestLoss: 2.3021793 TestAcc: 0.12
TrainLoss: 2.304102 TrainAcc: 0.04 Te

TrainLoss: 2.3033116 TrainAcc: 0.1 TestLoss: 2.3027604 TestAcc: 0.12
TrainLoss: 2.3015966 TrainAcc: 0.14 TestLoss: 2.3011703 TestAcc: 0.14
TrainLoss: 2.2997177 TrainAcc: 0.18 TestLoss: 2.298987 TestAcc: 0.14
TrainLoss: 2.3034906 TrainAcc: 0.08 TestLoss: 2.301183 TestAcc: 0.1
TrainLoss: 2.3023713 TrainAcc: 0.1 TestLoss: 2.3021727 TestAcc: 0.1
TrainLoss: 2.301507 TrainAcc: 0.1 TestLoss: 2.304729 TestAcc: 0.08
TrainLoss: 2.3032928 TrainAcc: 0.06 TestLoss: 2.305436 TestAcc: 0.04
TrainLoss: 2.3021286 TrainAcc: 0.14 TestLoss: 2.3015113 TestAcc: 0.08
TrainLoss: 2.3012106 TrainAcc: 0.04 TestLoss: 2.3033946 TestAcc: 0.08
TrainLoss: 2.3069456 TrainAcc: 0.06 TestLoss: 2.3035843 TestAcc: 0.08
TrainLoss: 2.3042521 TrainAcc: 0.06 TestLoss: 2.3038397 TestAcc: 0.12
TrainLoss: 2.303671 TrainAcc: 0.08 TestLoss: 2.3028286 TestAcc: 0.06
TrainLoss: 2.3042138 TrainAcc: 0.1 TestLoss: 2.302657 TestAcc: 0.08
TrainLoss: 2.3016481 TrainAcc: 0.16 TestLoss: 2.3031464 TestAcc: 0.06
TrainLoss: 2.3033807 TrainAcc: 0.

TrainLoss: 2.304303 TrainAcc: 0.08 TestLoss: 2.301332 TestAcc: 0.18
TrainLoss: 2.3013074 TrainAcc: 0.16 TestLoss: 2.302722 TestAcc: 0.12
TrainLoss: 2.3038225 TrainAcc: 0.1 TestLoss: 2.3049629 TestAcc: 0.06
TrainLoss: 2.3024669 TrainAcc: 0.1 TestLoss: 2.3011017 TestAcc: 0.06
TrainLoss: 2.303963 TrainAcc: 0.16 TestLoss: 2.302149 TestAcc: 0.1
TrainLoss: 2.2995539 TrainAcc: 0.14 TestLoss: 2.3042178 TestAcc: 0.06
TrainLoss: 2.304335 TrainAcc: 0.06 TestLoss: 2.3005128 TestAcc: 0.16
TrainLoss: 2.2959816 TrainAcc: 0.26 TestLoss: 2.304073 TestAcc: 0.08
TrainLoss: 2.2953653 TrainAcc: 0.2 TestLoss: 2.3018882 TestAcc: 0.1
TrainLoss: 2.301112 TrainAcc: 0.14 TestLoss: 2.302732 TestAcc: 0.1
TrainLoss: 2.3040373 TrainAcc: 0.08 TestLoss: 2.3005497 TestAcc: 0.1
TrainLoss: 2.3026175 TrainAcc: 0.1 TestLoss: 2.303595 TestAcc: 0.08
TrainLoss: 2.3040817 TrainAcc: 0.06 TestLoss: 2.3053603 TestAcc: 0.08
TrainLoss: 2.3018897 TrainAcc: 0.14 TestLoss: 2.3008912 TestAcc: 0.12
TrainLoss: 2.304385 TrainAcc: 0.06 Tes

TrainLoss: 2.3037477 TrainAcc: 0.02 TestLoss: 2.3050344 TestAcc: 0.06
TrainLoss: 2.3031049 TrainAcc: 0.04 TestLoss: 2.3037362 TestAcc: 0.06
TrainLoss: 2.3028324 TrainAcc: 0.1 TestLoss: 2.3038862 TestAcc: 0.12
TrainLoss: 2.3023715 TrainAcc: 0.12 TestLoss: 2.3033094 TestAcc: 0.08
TrainLoss: 2.3036523 TrainAcc: 0.12 TestLoss: 2.3015559 TestAcc: 0.16
TrainLoss: 2.3048925 TrainAcc: 0.02 TestLoss: 2.3000555 TestAcc: 0.1
TrainLoss: 2.3054411 TrainAcc: 0.08 TestLoss: 2.3021493 TestAcc: 0.08
TrainLoss: 2.3003564 TrainAcc: 0.14 TestLoss: 2.3042002 TestAcc: 0.08
TrainLoss: 2.3028088 TrainAcc: 0.08 TestLoss: 2.3034399 TestAcc: 0.08
TrainLoss: 2.3050208 TrainAcc: 0.04 TestLoss: 2.3042786 TestAcc: 0.06
TrainLoss: 2.3012817 TrainAcc: 0.04 TestLoss: 2.3022006 TestAcc: 0.14
TrainLoss: 2.3025343 TrainAcc: 0.08 TestLoss: 2.3034992 TestAcc: 0.08
TrainLoss: 2.301013 TrainAcc: 0.1 TestLoss: 2.3043642 TestAcc: 0.08
TrainLoss: 2.3022108 TrainAcc: 0.12 TestLoss: 2.304734 TestAcc: 0.1
TrainLoss: 2.3037393 Train

TrainLoss: 2.3057055 TrainAcc: 0.06 TestLoss: 2.3077796 TestAcc: 0.08
TrainLoss: 2.303618 TrainAcc: 0.08 TestLoss: 2.3045163 TestAcc: 0.1
TrainLoss: 2.302879 TrainAcc: 0.1 TestLoss: 2.3007166 TestAcc: 0.08
TrainLoss: 2.3019202 TrainAcc: 0.12 TestLoss: 2.3027692 TestAcc: 0.12
TrainLoss: 2.30247 TrainAcc: 0.08 TestLoss: 2.3016043 TestAcc: 0.06
TrainLoss: 2.3008282 TrainAcc: 0.1 TestLoss: 2.3036795 TestAcc: 0.08
TrainLoss: 2.3007903 TrainAcc: 0.12 TestLoss: 2.3020775 TestAcc: 0.08
TrainLoss: 2.3044245 TrainAcc: 0.08 TestLoss: 2.3017116 TestAcc: 0.08
TrainLoss: 2.302137 TrainAcc: 0.08 TestLoss: 2.3009028 TestAcc: 0.12
TrainLoss: 2.301969 TrainAcc: 0.18 TestLoss: 2.3001575 TestAcc: 0.14
TrainLoss: 2.304138 TrainAcc: 0.08 TestLoss: 2.3007555 TestAcc: 0.16
TrainLoss: 2.2993894 TrainAcc: 0.16 TestLoss: 2.3026452 TestAcc: 0.1
TrainLoss: 2.3046784 TrainAcc: 0.02 TestLoss: 2.3044634 TestAcc: 0.12
TrainLoss: 2.3041222 TrainAcc: 0.06 TestLoss: 2.2981663 TestAcc: 0.1
TrainLoss: 2.3005106 TrainAcc: 0

Observation: Because Gradients are not calculated as frequently as using Keras, accuracy is very low